In [2]:
import findspark
findspark.init()

In [3]:
import pyspark

from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer, VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("").getOrCreate()
#from pyspark.python.pyspark.shell import spark

your 131072x1 screen size is bogus. expect trouble
23/04/25 10:26:38 WARN Utils: Your hostname, WeebMachine resolves to a loopback address: 127.0.1.1; using 172.30.248.116 instead (on interface eth0)
23/04/25 10:26:38 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/04/25 10:26:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [33]:
data = spark.read.load("Absenteeism_at_work.csv", format="csv", header=True, delimiter=",")

data = data.withColumn("MOA", data["Month of absence"] - 0).withColumn("label", data['Transportation expense'] - 0). \
    withColumn("ROA", data["Reason for absence"] - 0). \
    withColumn("distance", data["Distance from Residence to Work"] - 0). \
    withColumn("BMI", data["Body mass index"] - 0)

# Index labels, adding metadata to the label column.
# Fit on whole dataset to include all labels in index.

vassembler = VectorAssembler(inputCols=["MOA", "ROA","distance", "BMI"], outputCol='features')

data = vassembler.transform(data)

labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(data)

# Automatically identify categorical features, and index them.
# Set maxCategories so features with > 4 distinct values are treated as continuous.
featureIndexer =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(data)

# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = data.randomSplit([0.7, 0.3])

# Train a RandomForest model.
rf = RandomForestClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures", numTrees=10)

# Convert indexed labels back to original labels.
labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel",
                               labels=labelIndexer.labels)

# Chain indexers and forest in a Pipeline
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, rf, labelConverter])

# Train model.  This also runs the indexers.
model = pipeline.fit(trainingData)

# Make predictions.
predictions = model.transform(testData)
#get y_true and y_pred
y_true = predictions.select("indexedLabel").rdd.flatMap(lambda x: x).collect()
y_pred = predictions.select("prediction").rdd.flatMap(lambda x: x).collect()

# Select example rows to display.
predictions.select("predictedLabel", "label", "features").show(5)

# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")

accuracy = evaluator.evaluate(predictions)

confusionmatrix = confusion_matrix(y_true, y_pred)

precision = precision_score(y_true, y_pred, average='micro')

recall = recall_score(y_true, y_pred, average='micro')

rfModel = model.stages[2]
print(rfModel)  # summary only
print("Random Forest - Test Accuracy = %g" % (accuracy))
print("Random Forest - Test Error = %g" % (1.0 - accuracy))

print("The Confusion Matrix for Random Forest Model is :\n" + str(confusionmatrix))

print("The precision score for Random Forest Model is: " + str(precision))

print("The recall score for Random Forest Model is: " + str(recall))

+--------------+-----+--------------------+
|predictedLabel|label|            features|
+--------------+-----+--------------------+
|         235.0|235.0|[1.0,13.0,11.0,29.0]|
|         235.0|235.0|[8.0,19.0,11.0,29.0]|
|         235.0|235.0|[8.0,23.0,11.0,29.0]|
|         235.0|235.0|[11.0,25.0,11.0,2...|
|         361.0|361.0|[8.0,13.0,52.0,27.0]|
+--------------+-----+--------------------+
only showing top 5 rows

RandomForestClassificationModel: uid=RandomForestClassifier_8836cdd844ab, numTrees=10, numClasses=24, numFeatures=4
Random Forest - Test Accuracy = 0.918919
Random Forest - Test Error = 0.0810811
The Confusion Matrix for Random Forest Model is :
[[50  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0 24  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  2 22  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0 11  1  0  0  0  0  0  0  5  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0 15  0  0  0  0  0  0  0  0  0  0  0  0  0  

In [5]:
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.evaluation import RegressionEvaluator

In [32]:
# Load the data as a DataFrame.
#data = spark.read.load("winequality-red.csv", format="csv", header=True, delimiter=";")

data = spark.read.format("csv").load("winequality-red.csv", header=True, delimiter=";")

data = data.withColumn("FA", data["fixed acidity"] - 0).withColumn("label", data['quality'] - 0). \
    withColumn("CA", data["citric acid"] - 0). \
    withColumn("RS", data["residual sugar"] - 0). \
    withColumn("A", data["alcohol"] - 0)

vassembler = VectorAssembler(inputCols=["FA", "CA", "RS","A"], outputCol='features')

data = vassembler.transform(data)

# Automatically identify categorical features, and index them.
# We specify maxCategories so features with > 4 distinct values are treated as continuous.
featureIndexer =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(data)

# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = data.randomSplit([0.7, 0.3])

# Train a DecisionTree model.
dt = DecisionTreeRegressor(featuresCol="indexedFeatures")

# Chain indexer and tree in a Pipeline
pipeline = Pipeline(stages=[featureIndexer, dt])

# Train model.  This also runs the indexer.
model = pipeline.fit(trainingData)

# Make predictions.
predictions = model.transform(testData)

# Select example rows to display.
predictions.select("prediction", "label", "features").show(5)

# Select (prediction, true label) and compute test error
evaluator_rmse = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator_rmse.evaluate(predictions)

evaluator_r2 = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName= "r2")
r2 = evaluator_r2.evaluate(predictions)

print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)
print("Coefficient of Determination (R^2) on test data = %g" % r2)

treeModel = model.stages[1]
# summary only
print(treeModel)

+-----------------+-----+--------------------+
|       prediction|label|            features|
+-----------------+-----+--------------------+
|             6.76|  6.0|[10.0,0.47,2.0,12.0]|
|             6.76|  6.0| [10.0,0.5,3.4,11.8]|
|             6.76|  7.0|[10.0,0.49,2.7,11.6]|
|5.675324675324675|  6.0|[10.0,0.24,2.2,10.1]|
|5.242761692650334|  5.0| [10.0,0.22,1.9,9.5]|
+-----------------+-----+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data = 0.72788
Coefficient of Determination (R^2) on test data = 0.137505
DecisionTreeRegressionModel: uid=DecisionTreeRegressor_ce07a30d9cd0, depth=5, numNodes=55, numFeatures=4


In [7]:
from pyspark.ml.regression import RandomForestRegressor

In [31]:
# Load the data as a DataFrame.
#data = spark.read.load("winequality-red.csv", format="csv", header=True, delimiter=";")

data = spark.read.format("csv").load("winequality-red.csv", header=True, delimiter=";")

data = data.withColumn("FA", data["fixed acidity"] - 0).withColumn("label", data['quality'] - 0). \
    withColumn("CA", data["citric acid"] - 0). \
    withColumn("RS", data["residual sugar"] - 0). \
    withColumn("A", data["alcohol"] - 0)

vassembler = VectorAssembler(inputCols=["FA", "CA", "RS","A"], outputCol='features')

data = vassembler.transform(data)

# Automatically identify categorical features, and index them.
# Set maxCategories so features with > 4 distinct values are treated as continuous.
featureIndexer =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(data)

# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = data.randomSplit([0.7, 0.3])

# Train a RandomForest model.
rf = RandomForestRegressor(featuresCol="indexedFeatures")

# Chain indexer and forest in a Pipeline
pipeline = Pipeline(stages=[featureIndexer, rf])

# Train model.  This also runs the indexer.
model = pipeline.fit(trainingData)

# Make predictions.
predictions = model.transform(testData)

# Select example rows to display.
predictions.select("prediction", "label", "features").show(5)

# Select (prediction, true label) and compute test error
evaluator_rmse = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator_rmse.evaluate(predictions)

evaluator_r2 = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName= "r2")
r2 = evaluator_r2.evaluate(predictions)

print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)
print("Coefficient of Determination (R^2) on test data = %g" % r2)

rfModel = model.stages[1]
print(rfModel)  # summary only

+------------------+-----+--------------------+
|        prediction|label|            features|
+------------------+-----+--------------------+
| 6.403534847031705|  8.0|[10.0,0.54,1.9,11.8]|
|5.5320667038436895|  5.0|  [10.0,0.4,2.9,9.7]|
| 5.345575832925095|  5.0| [10.0,0.45,2.5,9.4]|
|5.3143568745341065|  5.0| [10.0,0.38,1.6,8.5]|
| 6.510340505388244|  6.0| [10.0,0.5,3.4,11.8]|
+------------------+-----+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data = 0.671773
Coefficient of Determination (R^2) on test data = 0.336002
RandomForestRegressionModel: uid=RandomForestRegressor_c95e40695dbf, numTrees=20, numFeatures=4
